<img src="https://i.postimg.cc/5NKS0rfx/nn.png" alt="NN" width="500"/>


In [1]:
import numpy as np

In [49]:
class NeuralNetwork():

    # Set random seed
    np.random.seed(42)
# output_act_ -> activation function // by default is linear function
# error_fn -> error estimation function // by default mean square error
    def __init__(self, X, y, n_hidden_neurons, output_act_fn = 'linear', error_fn = 'mse'):
        self.X = X
        self.y = y
        self.n_input_neurons = X.shape[1] # assumed one hidden layer(shallow NN) for simplicity
        self.n_hidden_neurons = n_hidden_neurons
        self.output_act_fn = output_act_fn
        self.error_fn = error_fn

        # Initialize weights and biases with random values (array of random numbers)
        self.input_hidden_weights = np.random.randn(self.n_input_neurons, self.n_hidden_neurons)
        self.hidden_biases = np.random.randn(self.n_hidden_neurons)
       # 1 is the number of output neurons
        self.hidden_output_weights = np.random.randn(self.n_hidden_neurons, 1)
        self.output_bias = np.random.randn(1)

    def activation(self, x, act_fn):
        if act_fn == 'sigmoid':
            return 1 / (1 + np.exp(-x))
        elif act_fn == 'relu':
            return np.maximum(0, x)
        elif act_fn == 'linear':
            return x
        else:
            raise Exception('Unknown activation function')

    def activation_derivative(self, x, act_fn):
        if act_fn == 'sigmoid':
            return x * (1 - x)
        elif act_fn == 'relu':
            return np.where(x > 0, 1, 0)
        elif act_fn == 'linear':
            return 1
        else:
            raise Exception('Unknown activation function')

    # Forward Propagation
    def forward_pass(self, X):
        # Input layer
        self.input = X

        # Hidden layer
        self.hidden = self.activation(np.dot(self.input, self.input_hidden_weights) + self.hidden_biases, 'relu')

        # Output layer
        self.output = self.activation(np.dot(self.hidden, self.hidden_output_weights) + self.output_bias, self.output_act_fn)

        return self.output

    # Error Estimation
    def error(self, y_true, y_pred):
        if self.error_fn == 'mse':
            return np.mean(np.square(y_true - y_pred))
        elif self.error_fn == 'cross_entropy':
            return -np.mean(y_true * np.log(y_pred+0.00001) + (1 - y_true) * np.log(1 - y_pred+0.00001))
        else:
            raise Exception('Unknown error function')

    def error_derivative(self, y_true, y_pred):
        if self.error_fn == 'mse':
            return  2 * (y_pred - y_true) / y_true.size
        elif self.error_fn == 'cross_entropy':
            return (y_pred - y_true) / (y_pred * (1 - y_pred+.00001) * y_true.size)
        else:
            raise Exception('Unknown error function')

    # Backpropagation
    def backward_pass(self, X, y_true, y_pred, learning_rate):
        # Output layer
        self.output_error = self.error_derivative(y_true, y_pred) * self.activation_derivative(y_pred, self.output_act_fn)
        self.output_bias -= learning_rate * np.sum(self.output_error, axis=0)
        self.hidden_output_weights -= learning_rate * np.dot(self.hidden.T, self.output_error)

        # Hidden layer
        self.hidden_error = np.dot(self.output_error, self.hidden_output_weights.T) * self.activation_derivative(self.hidden, 'relu')
        self.hidden_biases -= learning_rate * np.sum(self.hidden_error, axis=0)
        self.input_hidden_weights -= learning_rate * np.dot(X.T, self.hidden_error)

        # return weights and biases
        return self.input_hidden_weights, self.hidden_biases, self.hidden_output_weights, self.output_bias

    # Training
    def train(self, X, y, learning_rate, epochs):
        for epoch in range(epochs):
            y_pred = self.forward_pass(X)
            wih, bh, who, bo = self.backward_pass(X, y, y_pred, learning_rate)
            if epoch % 500 == 0:
                print('Epoch: {}, Loss: {:.3f}'.format(epoch, self.error(y, y_pred)))
        print('Training complete!')
        return wih, bh, who, bo

    # Prediction
    def predict(self, X):
        if self.error_fn == 'mse':
            return self.forward_pass(X)
        elif self.error_fn == 'cross_entropy':
            return np.where(self.forward_pass(X) > 0.5, 1, 0)


**Regression Example**

In [46]:
from sklearn.datasets import make_regression
X, y = make_regression(n_samples=1000, n_features=3, noise=20, random_state=42)
y = y.reshape(-1, 1)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [48]:
nn = NeuralNetwork(X_train, y_train,  n_hidden_neurons=128, output_act_fn='linear', error_fn='mse')
wih, bh, who, bo= nn.train(X_train, y_train, learning_rate=0.001, epochs=5000)

Epoch: 0, Loss: 13892.784
Epoch: 500, Loss: 374.455
Epoch: 1000, Loss: 367.597
Epoch: 1500, Loss: 362.922
Epoch: 2000, Loss: 359.279
Epoch: 2500, Loss: 356.310
Epoch: 3000, Loss: 354.433
Epoch: 3500, Loss: 352.962
Epoch: 4000, Loss: 351.569
Epoch: 4500, Loss: 350.323
Training complete!


In [50]:
# Weights and biases
print('Input-hidden weights:\n', wih)
print('Hidden-output weights:\n', who)
print('Hidden biases:\n', bh)
print('Output bias:\n', bo)

Input-hidden weights:
 [[ 0.46218215  0.32542526  0.16580122  1.95488085  0.38762215 -0.20067982
   2.37748297  1.29989384 -2.04491936  0.99280742 -1.00687122 -0.45065981
   0.24196227 -2.53614614 -2.86502429 -0.57946922 -0.97141953  0.43746702
  -1.1375737  -2.10772852  1.68217445 -1.30138492  0.0675282  -1.46967514
  -0.85697598  0.04103586 -0.18329947  0.27823312  0.29107773 -1.26135854
  -0.29820138  2.74711965 -0.94704823 -1.47916521  1.36182059 -1.38830393
  -0.14503344 -2.32309602 -1.27638632  0.53413518  1.37159347  0.25528276
   0.06449208 -0.4329507  -2.22006145 -0.54760222 -0.35777331  0.84362734
   0.56385418 -0.65745027  0.81709272 -0.34187398 -0.38302591  0.90215247
   1.3512      0.82033207 -2.03080346 -0.66764091  0.45827357  2.0147918
  -0.42215082 -0.18260957 -0.91808413 -1.86473578  1.12831257  1.94606514
  -1.14753947  0.31739748  0.58125847 -1.36779867  1.01385854  3.32589176
   0.17883453  2.03555612 -3.01400746  0.62750092  0.13721074 -0.990968
   0.81978882 -2.1

In [51]:
# Accuracy
from sklearn.metrics import r2_score
y_pred = nn.predict(X_test)
print('R2 Score: {:.3f}'.format(r2_score(y_test, y_pred)))

R2 Score: 0.975


In [52]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
print('R2 Score: {:.3f}'.format(r2_score(y_test, y_pred)))

R2 Score: 0.976


In [54]:
# MlpRegressor
from sklearn.neural_network import MLPRegressor
mlp = MLPRegressor(hidden_layer_sizes=(128,), activation='relu', solver='adam', max_iter=5000, random_state=42)
y_train = y_train.reshape(-1)
mlp.fit(X_train, y_train)

MLPRegressor(hidden_layer_sizes=(128,), max_iter=5000, random_state=42)

In [55]:
y_train.shape

(750,)

In [56]:
y_pred = mlp.predict(X_test)
y_test = y_test.reshape(-1)
print('R2 Score: {:.3f}'.format(r2_score(y_test, y_pred)))

R2 Score: 0.976


In [57]:
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=1000, n_features=10, n_informative=5, n_redundant=5, random_state=42)
y = y.reshape(-1, 1)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [58]:
nn = NeuralNetwork(X_train, y_train,  n_hidden_neurons=128, output_act_fn='sigmoid', error_fn='cross_entropy')
wih, bh, who, bo = nn.train(X_train, y_train, learning_rate=0.001, epochs=5000)

Epoch: 0, Loss: 5.929
Epoch: 500, Loss: 4.358
Epoch: 1000, Loss: 2.837
Epoch: 1500, Loss: 0.827
Epoch: 2000, Loss: 0.657
Epoch: 2500, Loss: 0.580
Epoch: 3000, Loss: 0.525
Epoch: 3500, Loss: 0.486
Epoch: 4000, Loss: 0.454
Epoch: 4500, Loss: 0.424
Training complete!


In [59]:
print('Input-hidden weights:\n', wih)
print('Hidden-output weights:\n', who)
print('Hidden biases:\n', bh)
print('Output bias:\n', bo)

Input-hidden weights:
 [[ 4.91183828e-01 -1.00468647e-01  6.44106378e-01 ...  2.18844993e+00
  -9.56418207e-01 -4.09697867e-01]
 [ 9.15288413e-02 -4.90635188e-01 -1.53993291e+00 ...  1.04335534e+00
  -1.47839834e+00 -4.42668812e-01]
 [ 1.26456992e+00 -7.14245936e-01  4.96105956e-01 ... -6.09409034e-01
  -2.14842526e+00 -5.82169478e-01]
 ...
 [-1.53604276e-01  1.68445391e-01  1.13970488e+00 ... -5.75899104e-01
  -2.39451060e-02  2.23720422e+00]
 [ 1.72978081e+00  4.83170695e-01 -1.35860334e-03 ... -8.48581350e-01
  -9.56506553e-01 -1.96117134e+00]
 [-6.46226342e-01 -1.31219010e+00  1.66662767e+00 ...  3.94603584e-01
   2.12066826e-01 -2.76227811e-01]]
Hidden-output weights:
 [[-2.62953365e-01]
 [-3.29512105e-01]
 [ 1.20847033e+00]
 [ 3.13681443e-01]
 [ 3.67716485e-01]
 [ 2.14878029e+00]
 [-9.06006210e-01]
 [ 7.28395787e-01]
 [ 2.49850589e-01]
 [ 1.79070340e-01]
 [ 1.62902030e+00]
 [ 1.50191246e-01]
 [ 6.30643759e-01]
 [ 7.44890741e-01]
 [-3.55293161e-01]
 [-1.16744705e+00]
 [-1.11634281

In [60]:
y_pred = nn.predict(X_test)

from sklearn.metrics import accuracy_score
print('Accuracy: {:.3f}'.format(accuracy_score(y_test, y_pred)))

Accuracy: 0.908


In [61]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
y_train = y_train.reshape(-1)

lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)
print('Accuracy: {:.3f}'.format(accuracy_score(y_test, y_pred)))

Accuracy: 0.792


In [62]:
# MLPClassifier
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(128,), activation='relu', solver='adam', max_iter=5000, random_state=42)
mlp.fit(X_train, y_train)

y_pred = mlp.predict(X_test)
print('Accuracy: {:.3f}'.format(accuracy_score(y_test, y_pred)))

Accuracy: 0.940
